# 📦 12_load_playlists_manual_static_to_bigquery

### 🎯 Objetivo

Este notebook no genera snapshots históricos.

Su propósito es cargar la tabla estructural playlists_manual_static a BigQuery para permitir joins analíticos con las tablas snapshot.

Esta tabla actúa como dimensión descriptiva de playlists manuales y se reescribe completamente en cada ejecución del pipeline mensual.  

Destino: youtube-datasets-360.angelgarciadatablog.playlists_manual_static

In [3]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery

In [4]:
load_dotenv()

PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "playlists_manual_static"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)


Destino configurado: youtube-datasets-360.angelgarciadatablog.playlists_manual_static


## 🧱 Cargar snapshot desde Parquet (temporal)  

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [5]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

df_playlists_manual_static = pd.read_parquet(
    PROCESSED_PATH / "playlists_manual_static.parquet"
)

df_playlists_manual_static.head()


,playlist_id,channel_id,title,description,item_count,privacy_status,published_at,thumbnail_url,playlist_url,extracted_at
0,PLV4oS06_KpqbsY_I8iR4HRvb6w3vXUBIM,UCUEOHBht8pnQhQvCfIcl-gg,SQL - Repaso,,2,public,2026-01-23 05:43:30.512825+00:00,https://i.ytimg.com/vi/7bwkNrRpgw0/hqdefault.jpg,https://www.youtube.com/playlist?list=PLV4oS06...,2026-02-16 06:55:49.536537+00:00
1,PLV4oS06_KpqZGwOHo-tsdIiaZts7qaqql,UCUEOHBht8pnQhQvCfIcl-gg,Python - Repaso,,3,public,2026-01-11 23:05:18.079131+00:00,https://i.ytimg.com/vi/Zj6uiqMvFOU/hqdefault.jpg,https://www.youtube.com/playlist?list=PLV4oS06...,2026-02-16 06:55:49.536537+00:00
2,PLV4oS06_KpqaqyS9x6h5ys3REiUfUDOgy,UCUEOHBht8pnQhQvCfIcl-gg,Curso gratuito de SQL en BigQuery | Funciones ...,Aprende a realizar cálculos avanzados en SQL s...,5,public,2026-01-11 22:39:38.064293+00:00,https://i.ytimg.com/vi/XHnA-a5rivU/hqdefault.jpg,https://www.youtube.com/playlist?list=PLV4oS06...,2026-02-16 06:55:49.536537+00:00
3,PLV4oS06_KpqbhnVieDd19KJczH_BlBArN,UCUEOHBht8pnQhQvCfIcl-gg,Git - Repaso,,4,public,2026-01-11 10:10:50.096597+00:00,https://i.ytimg.com/vi/GLJffh1QDM4/hqdefault.jpg,https://www.youtube.com/playlist?list=PLV4oS06...,2026-02-16 06:55:49.536537+00:00
4,PLV4oS06_KpqYRtYRoQHo_F_KsEjmqcDK7,UCUEOHBht8pnQhQvCfIcl-gg,Power Bi - Repaso,Aprende funciones DAX avanzadas aplicadas a ca...,14,public,2025-11-14 07:55:25.117707+00:00,https://i.ytimg.com/vi/J0GOADsEp4E/hqdefault.jpg,https://www.youtube.com/playlist?list=PLV4oS06...,2026-02-16 06:55:49.536537+00:00


In [6]:
df_playlists_manual_static.dtypes

playlist_id                    object
channel_id                     object
title                          object
description                    object
item_count                      int64
privacy_status                 object
published_at      datetime64[ns, UTC]
thumbnail_url                  object
playlist_url                   object
extracted_at      datetime64[ns, UTC]
dtype: object

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 

En este caso definimos el esquema manualmente en lugar de dejar que BigQuery lo infiera automáticamente desde el DataFrame. Esto nos permite controlar explícitamente los tipos de datos, especialmente published_at y extracted_at, que queremos almacenar como DATETIME (hora local Perú) y no como TIMESTAMP. Cuando BigQuery infiere el esquema, puede convertir columnas datetime en TIMESTAMP, lo que introduce interpretación en UTC. Definir el esquema manualmente garantiza consistencia y evita ambigüedades en el modelo temporal.

In [7]:
from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("playlist_id", "STRING"),
    SchemaField("channel_id", "STRING"),
    SchemaField("title", "STRING"),
    SchemaField("description", "STRING"),
    SchemaField("item_count", "INT64"),
    SchemaField("privacy_status", "STRING"),
    SchemaField("published_at", "TIMESTAMP"),
    SchemaField("thumbnail_url", "STRING"),
    SchemaField("playlist_url", "STRING"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("La tabla ya existe.")

except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)
    client.create_table(table)
    print("Tabla creada correctamente.")



Tabla creada correctamente.


## 📌 Cargar datos del parquet a big query

In [8]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE TRUNCATE = sobreescribe los datos
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df_playlists_manual_static,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Tabla playlists_manual_static reemplazada correctamente.")



Tabla playlists_manual_static reemplazada correctamente.
